<div style="text-align: center;">
    <h1><strong>UNIVERSIDAD POLITÉCNICA DE CARTAGENA</strong></h1>
    <h2><strong>GRADO EN CIENCIA E INGENIERÍA DE DATOS</strong></h2>

<div style="display:flex; justify-content:center; align-items:center; padding:5px;">
        <img src="imgs/upct_logo.png" style="height:300px; width:auto">
    </div>

<h2><strong>Procesamiento del Lenguaje Natural</strong></h2>

<h3><strong>PROYECTO:</strong><br>
    <strong>Aplicación PLN para análisis y procesamiento de texto de carácter geopolítico, económico-bursátil y político.</strong></h3>

<p><strong>ESTUDIANTES</strong></p>
    <ul style="list-style-type:none; padding: 0;">
        <li>Rubén Gil Martínez</li>
        <li>Guillermo López Pérez</li>
    </ul>
</div>


### **1) COMPILACIÓN DEL CORPUS DE COMENTARIOS DE REDDIT A TRAVÉS DE SU API:**

**LIBRERÍAS**

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import datetime as dt
import time

import praw
import pandas as pd
from datetime import datetime
import time
import re
import os

**EXTRACCIÓN DEL CORPUS DE COMENTARIOS EN ARCHIVOS JSON INDIVIDUALES PARA CADA SUBREDDIT:**

In [3]:
# Configuración de credenciales
reddit = praw.Reddit(
    client_id='v4Ozy7so3ZR5n-i4r7Spqw',
    client_secret='iDAtL0KxYY9ky4GyPWyUM2OAZ4LnBg',
    user_agent='dl-pln-2025-RGM-GLP'
)

In [ ]:
# Prueba simple de correcta autenticación
try:
    # Intenta acceder a tu nombre de usuario para verificar autenticación
    print(f"Autenticado como: {reddit.user.me()}")
    
    # Prueba una solicitud simple
    for submission in reddit.subreddit("test").hot(limit=1):
        print(f"Título del post de prueba: {submission.title}")
        
    print("¡Autenticación exitosa!")
    
except Exception as e:
    print(f"Error de autenticación: {type(e).__name__} - {str(e)}")
    print("Por favor, verifica tus credenciales y la configuración de la aplicación.")

Autenticado como: None
Título del post de prueba: Some test commands
¡Autenticación exitosa!


In [5]:
# Lista de subreddits a extraer
subreddits = ['Economics', 'finance', 'StockMarket', 'geopolitics', 'worldnews', 'business']

# Expresiones regulares para filtrado
url_pattern = re.compile(r'(https?://\S+)')
email_pattern = re.compile(r'[\w\.-]+@[\w\.-]+')

# Función para verificar si un comentario es válido
def is_valid_comment(text):
    if len(text) < 30:  # Mínimo de 30 caracteres
        return False
    
    # Eliminar URLs y emails
    text_cleaned = url_pattern.sub(' ', text)
    text_cleaned = email_pattern.sub(' ', text_cleaned)
    
    # Verificar si queda suficiente contenido
    return len(text_cleaned.strip()) >= 30


# Procesamiento de cada subreddit
for subreddit_name in subreddits:
    print(f"Procesando subreddit: {subreddit_name}...")
    
    subreddit_data = {
        "subreddit_name": subreddit_name,
        "threads": []
    }

    subreddit = reddit.subreddit(subreddit_name)
    threads_collected = 0  # Contador de hilos extraídos
    extracted_thread_ids = set()  # Para evitar duplicados

    # Diferentes filtros de tiempo
    time_filters = ['all', 'week', 'month', 'year']

    for time_filter in time_filters:
        if threads_collected >= 20:
            break
        
        print(f"  Extrayendo de {subreddit_name} con filtro: {time_filter}")

        try:
            threads = list(subreddit.top(time_filter=time_filter, limit=8))  # Procesamos 8 hilos para cada temporalidad para mayor rapidez y evitar bloqueos
            
            for thread in threads:
                if threads_collected >= 20 or thread.id in extracted_thread_ids:  # Evitar duplicados
                    continue  

                print(f"    Procesando hilo: {thread.title}")

                # Información del hilo
                thread_info = {
                    "thread_id": thread.id,
                    "title": thread.title,
                    "author": str(thread.author),
                    "created_utc": datetime.utcfromtimestamp(thread.created_utc).isoformat(),
                    "score": thread.score,
                    "url": thread.url,
                    "permalink": thread.permalink,
                    "time_filter": time_filter,
                    "subreddit_name": subreddit_name,
                    "comments": []
                }


                # Extraer comentarios
                try:
                    thread.comments.replace_more(limit=3)  # Evita cargar demasiados comentarios
                    flat_comments = thread.comments.list()
                except Exception as e:
                    print(f"      Error al cargar comentarios: {e}")
                    continue

                valid_comments_count = 0

                for comment in flat_comments:
                    if valid_comments_count >= 50:
                        break

                    if is_valid_comment(comment.body):

                        comment_info = {
                            'comment_id': comment.id,
                            'author': str(comment.author),
                            'text': comment.body,
                            'score': comment.score,
                            'created_utc': datetime.utcfromtimestamp(comment.created_utc).isoformat(),
                            'subreddit_name': subreddit_name,
                            'thread_id': thread.id,
                            'is_submitter': comment.is_submitter,
                            'permalink': comment.permalink
                        }

                        thread_info["comments"].append(comment_info)
                        valid_comments_count += 1

                if valid_comments_count >= 25:  # Mínimo 25 comentarios válidos
                    subreddit_data["threads"].append(thread_info)
                    extracted_thread_ids.add(thread.id)
                    threads_collected += 1
                    print(f"      Añadido con {valid_comments_count} comentarios válidos")
                else:
                    print(f"      Descartado: solo {valid_comments_count} comentarios válidos")

                time.sleep(1.5)  # Evitar exceso de peticiones a Reddit

        except Exception as e:
            print(f"Error en {subreddit_name} con filtro {time_filter}: {e}")
            continue

    # Guardar en JSON
    os.makedirs('datasets', exist_ok=True)
    output_file = os.path.join('datasets', f"{subreddit_name}_data_2.0.json")

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(subreddit_data, f, ensure_ascii=False, indent=2)

    print(f"Completado: {subreddit_name} - {threads_collected} hilos guardados en {output_file}\n")

print("Extracción completa.")


Procesando subreddit: Economics...
  Extrayendo de Economics con filtro: all
    Procesando hilo: Economist Warns That Elon Musk Is About to Cause a "Deep, Deep Recession"
      Añadido con 50 comentarios válidos
    Procesando hilo: 65% of Americans support monthly $2,000 COVID stimulus payments, new poll shows
      Añadido con 50 comentarios válidos
    Procesando hilo: US tourism to suffer huge '£49 billion drop' under Donald Trump
      Añadido con 50 comentarios válidos
    Procesando hilo: Fed report finds 75% of $800 billion Paycheck Protection Program didn't reach employees
      Añadido con 30 comentarios válidos
    Procesando hilo: Trudeau announces Canada ban on Russian oil imports. “Today, we are announcing a ban on all imports of crude oil from Russia, an industry that has benefited President Putin and his oligarchs greatly,”
      Añadido con 50 comentarios válidos
    Procesando hilo: Trump's first 50 days mark one of the worst starts for the S&P 500 under any presiden

**DURANTE LA EXTRACCIÓN DE LOS COMENTARIOS SE HA TENIDO EN CUENTA LOS SIGUIENTES PUNTOS:**

- Verificación de comentarios válidos
- Distribución temporal diversa
- Descarte de hilos con pocos comentarios válidos



- Pausas para respetar límites de la API
- Manejo de excepciones para evitar interrupciones
- Seguimiento del progreso mediante mensajes

# **PREPROCESAMIENTO:**

In [7]:
# Leer el archivo JSON
with open('datasets/Economics_data_2.0.json', 'r') as file:
    economics = json.load(file)

# Mostrar el objeto JSON
print(economics)


{'subreddit_name': 'Economics', 'threads': [{'thread_id': '1iu0707', 'title': 'Economist Warns That Elon Musk Is About to Cause a "Deep, Deep Recession"', 'author': 'BothZookeepergame612', 'created_utc': '2025-02-20T14:49:46', 'score': 58546, 'url': 'https://futurism.com/economist-elon-musk-recession', 'permalink': '/r/Economics/comments/1iu0707/economist_warns_that_elon_musk_is_about_to_cause/', 'time_filter': 'all', 'subreddit_name': 'Economics', 'comments': [{'comment_id': 'mdt5caz', 'author': 'AutoModerator', 'text': 'Hi all,\n\nA reminder that comments do need to be on-topic and engage with the article past the headline. Please make sure to read the article before commenting.  Very short comments will automatically be removed by automod. Please avoid making comments that do not focus on the economic content or whose primary thesis rests on personal anecdotes. \n\nAs always our comment rules can be found [here](https://reddit.com/r/Economics/comments/fx9crj/rules_roundtable_redux_r

In [8]:
Economics_threads = pd.DataFrame(economics['threads'])
Economics_threads.head(5)  # Mostrar las primeras 5 filas del DataFrame
Economics_threads.info()  # Información general del DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   thread_id       20 non-null     object
 1   title           20 non-null     object
 2   author          20 non-null     object
 3   created_utc     20 non-null     object
 4   score           20 non-null     int64 
 5   url             20 non-null     object
 6   permalink       20 non-null     object
 7   time_filter     20 non-null     object
 8   subreddit_name  20 non-null     object
 9   comments        20 non-null     object
dtypes: int64(1), object(9)
memory usage: 1.7+ KB


In [10]:
for thread in range(len(Economics_threads)):
    Economics_comments = pd.DataFrame(economics['threads'][thread]['comments'])

In [13]:
print(len(Economics_threads))  # Número total de hilos
print(len(Economics_comments))  # Número total de comentarios

20
50


In [11]:
Economics_comments.head(5)  # Mostrar las primeras 5 filas del DataFrame

comment_id           author  \
0    lk5rdho    AutoModerator   
1    lk7u0kj  boxjellyfishing   
2    lk96bkt             None   
3    lk7nod0   Severe-Ad-8215   
4    lk5yqep        numbersev   

                                                text  score  \
0  Hi all,\n\nA reminder that comments do need to...      1   
1  While the Walton family is one of the richest ...    953   
2  Fuck yes they should.  Because if you don't ta...     97   
3  Why not just close up tax loopholes that allow...     29   
4  Panama Papers.\n\nThere’s basically a global c...    504   

           created_utc subreddit_name thread_id  is_submitter  \
0  2024-08-27T11:38:34      Economics   1f2eubo         False   
1  2024-08-27T18:43:52      Economics   1f2eubo         False   
2  2024-08-27T23:05:45      Economics   1f2eubo         False   
3  2024-08-27T18:10:55      Economics   1f2eubo         False   
4  2024-08-27T12:32:39      Economics   1f2eubo         False   

                                           permalink  
0  /r/Economics/comments/1f2eubo/should_the_world...  
1  /r/Economics/comments/1f2eubo/should_the_world...  
2  /r/Economics/comments/1f2eubo/should_the_world...  
3  /r/Economics/comments/1f2eubo/should_the_world...  
4  /r/Economics/comments/1f2eubo/should_the_world...